In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationEntityMemory
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import  ChatOpenAI
print(os.getenv('Deepseel_API_KEY'))
model=ChatOpenAI(model='deepseek-chat',api_key=os.getenv('Deepseel_API_KEY'),base_url='https://api.deepseek.com/v1')
from langchain.memory import ConversationKGMemory

: 

In [9]:
param=StrOutputParser()


#定义提示模板
prompt_template=ChatPromptTemplate.from_messages([
    ('human','请你作为一个大模型不进行任何检索回答下面的问题'),
    ('human','{text}')
])


#得到链
re=prompt_template|model|param
print(re.invoke({'text': '今天成都的天气怎么样'}))


APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

In [4]:
m=ConversationEntityMemory(llm=model)
_input={
    "input": "小明，小黄，小绿谁不一样",
}
m.load_memory_variables(_input)

{'history': '', 'entities': {'小明，小黄，小绿': ''}}

In [5]:
m.save_context(
_input,
{
    "output":"小明"
}

)

In [6]:
m.load_memory_variables({"input":"谁不一样"})

{'history': 'Human: 小明，小黄，小绿谁不一样\nAI: 小明',
 'entities': {'小明': '', '小黄': '', '小绿': ''}}

长对话保存的处理方式：总结摘要，以及计算token。

In [7]:
from langchain.memory import ConversationSummaryMemory
'''总结上下文形成摘要'''

m = ConversationSummaryMemory(llm=model)
m.save_context(
    {
        "input": "小明，小黄，小绿谁不一样",
    },
    {
        "output": "小明"
    }
)
m.save_context(
    {
        "input": "为什么小明不一样",
    },
    {
        "output": "因为小明不是颜色"
    }
)

m.load_memory_variables({})

C:\Users\yks\AppData\Local\Temp\ipykernel_11528\2197052748.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  m = ConversationSummaryMemory(llm=model)


{'history': 'The human asked the AI to identify differences among 小明, 小黄, and 小绿. The AI determined that 小明 is与众不同 because he is not a color.'}

In [8]:
mesage=m.chat_memory.messages
m.predict_new_summary(mesage,"")

'The human asked the AI to identify which one among Xiao Ming, Xiao Huang, and Xiao Lu is different. The AI pointed out that Xiao Ming is the与众不同 (different) one. When asked why, the AI explained that Xiao Ming is not a color, unlike Xiao Huang and Xiao Lu.'

In [13]:
from langchain.memory import ChatMessageHistory

history=ChatMessageHistory()
history.add_user_message('小明，小黄，小绿谁不一样')
history.add_ai_message('小明')

memory=ConversationSummaryMemory(
    llm=model,
    return_messages=True,
    chat_memory=history
)


In [14]:

memory.buffer

''

In [2]:
#对话持续进行且内容很多的适合 可以使用ConversationSummaryBufferMemory
#可以通过token的阈值进行自动摘要，比较久的优先进行摘要 目前通过报错来看阿里云的模型不支持计算 token数量
#目前这个类还无法使用

from langchain.memory import ConversationTokenBufferMemory
from openai import OpenAI
memory=ConversationTokenBufferMemory(
    llm=model,
    return_messages=True,
    max_token_limit=10
)
memory.save_context(
    {
        "input": "小明，小黄，小绿谁不一样",
    },
    {
        "output": "小明"
    }
)
memory.save_context(
    {
        "input": "为什么小明不一样",
    },
    {
        "output": "因为小明不是颜色"
    }
)
memory.save_context(
    {
        "input": "还因为什么？",
    },
    {
        "output": "因为小明笔画最少"
    }
)

C:\Users\yks\AppData\Local\Temp\ipykernel_9468\1172335075.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationTokenBufferMemory(


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI

memory = ConversationSummaryBufferMemory(
    llm=model,
    max_token_limit=400  # 触发摘要的 Token 阈值
)

# 模拟添加多轮对话（自动触发摘要）
for _ in range(10):
    memory.save_context({"input": "用户的问题"}, {"output": "模型的回答"})

# 查看当前内存内容
print(memory.load_memory_variables({}))

C:\Users\yks\AppData\Local\Temp\ipykernel_26792\4137319174.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(


NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

In [ ]:
from langchain.memory import ConversationBufferMemory
import tiktoken

# 初始化 LLM 和 Memory
llm =model
memory = ConversationBufferMemory()

# 初始化 Token 计算工具
encoding = tiktoken.get_encoding("cl100k_base") 

# 定义 Token 阈值
MAX_TOKEN_LIMIT = 100

def count_tokens(text):
    """统计文本的 Token 数量"""
    return len(encoding.encode(text))

def prune_memory_if_needed(memory):
    """检查并截断内存内容"""
    buffer = memory.load_memory_variables({})["history"]
    token_count = count_tokens(buffer)
    
    if token_count > MAX_TOKEN_LIMIT:
        # 截断逻辑：保留最近的对话
        messages = buffer.split("\n")
        while token_count > MAX_TOKEN_LIMIT and len(messages) > 1:
            messages.pop(0)  # 移除最早的消息
            buffer = "\n".join(messages)
            token_count = count_tokens(buffer)
        
        # 更新内存
        memory.clear()
        memory.save_context({"input": "截断历史"}, {"output": buffer})

# 模拟添加多轮对话
for i in range(50):
    memory.save_context({"input": f"用户问题 {i}"}, {"output": f"模型回答 {i}"})
    prune_memory_if_needed(memory)  # 检查并截断

# 查看当前内存内容
print(memory.load_memory_variables({}))

{'history': 'Human: 截断历史\nAI: AI: 模型回答 44\nHuman: 用户问题 45\nAI: 模型回答 45\nHuman: 用户问题 46\nAI: 模型回答 46\nHuman: 用户问题 47\nAI: 模型回答 47\nHuman: 用户问题 48\nAI: 模型回答 48\nHuman: 用户问题 49\nAI: 模型回答 49'}
